# Structured Outputs: Pydantic 

## 응답 형식 사용법

이전에는 `response_format` 매개변수가 모델이 유효한 JSON을 반환하도록 지정하는 데만 사용 가능했다.

이에 더해, 따라야 할 JSON 스키마를 지정하는 새로운 방법을 소개한다.

## 환경 설정

In [ ]:
%pip install -q pydantic eum

In [1]:
# 필요한 라이브러리들을 가져온다.
import json
from textwrap import dedent
from openai import OpenAI

import os
from dotenv import load_dotenv  

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI()

In [2]:
# 사용할 모델을 정의한다.
MODEL = "gpt-4.1-mini"

## 예제 1: 수학 교사

In [3]:
# 수학 교사를 위한 시스템 프롬프트를 정의한다.
math_tutor_prompt = '''
    당신은 도움이 되는 수학 교사다. 수학 문제가 주어지면,
    단계별 해결책과 최종 답을 출력하는 것이 목표다.
    각 단계에 대해서는 방정식으로 출력을 제공하고, 추론 과정을 자세히 설명하기 위해 설명 필드를 사용하라.
'''

# 수학 문제에 대한 해결책을 얻는 함수를 정의한다.
def get_math_solution(question):
    response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {
            "role": "system",
            "content": dedent(math_tutor_prompt) # dedent를 사용하여 들여쓰기를 제거한다.
        },
        {
            "role": "user",
            "content": question
        }
    ],
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "math_reasoning",
            "schema": {
                "type": "object",
                "properties": {
                    "steps": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "properties": {
                                "explanation": {"type": "string"},
                                "output": {"type": "string"}
                            },
                            "required": ["explanation", "output"],
                            "additionalProperties": False # 정의되지 않은 속성은 허용하지 않는다.
                        }
                    },
                    "final_answer": {"type": "string"}
                },
                "required": ["steps", "final_answer"],
                "additionalProperties": False
            },
            "strict": True # JSON 스키마를 엄격하게 따르도록 강제한다.
        }
    }
    )

    return response.choices[0].message

In [4]:
# 예제 질문으로 테스트한다.
question = "8x + 7 = -23 방정식을 어떻게 풀 수 있는가?"

result = get_math_solution(question)

# 응답 내용을 JSON 형식으로 출력한다.
print(result.content)

{"steps":[{"explanation":"주어진 방정식은 8x + 7 = -23입니다. 먼저 양변에서 7을 빼서 x가 있는 항을 분리합니다.","output":"8x + 7 - 7 = -23 - 7"},{"explanation":"좌변에서 7 - 7은 0이고, 우변에서 -23 - 7은 -30이므로 방정식은 8x = -30이 됩니다.","output":"8x = -30"},{"explanation":"이제 양변을 8로 나누어 x의 값을 구합니다.","output":"x = \\frac{-30}{8}"},{"explanation":"분수를 약분하면 x = -\\frac{15}{4}가 됩니다.","output":"x = -\\frac{15}{4}"}],"final_answer":"x = -\\frac{15}{4}"}


In [5]:
from IPython.display import Math, display

# 수학 응답을 예쁘게 출력하는 함수를 정의한다.
def print_math_response(response):
    result = json.loads(response)
    steps = result['steps']
    final_answer = result['final_answer']
    for i in range(len(steps)):
        print(f"단계 {i+1}: {steps[i]['explanation']}\\n")
        # display(Math(...))는 LaTeX 형식으로 수식을 렌더링한다.
        display(Math(steps[i]['output']))
        print("\\n")

    print("최종 답:\\n\\n")
    display(Math(final_answer))

In [6]:
# 포맷팅된 수학 응답을 출력한다.
print_math_response(result.content)

단계 1: 주어진 방정식은 8x + 7 = -23입니다. 먼저 양변에서 7을 빼서 x가 있는 항을 분리합니다.\n


<IPython.core.display.Math object>

\n
단계 2: 좌변에서 7 - 7은 0이고, 우변에서 -23 - 7은 -30이므로 방정식은 8x = -30이 됩니다.\n


<IPython.core.display.Math object>

\n
단계 3: 이제 양변을 8로 나누어 x의 값을 구합니다.\n


<IPython.core.display.Math object>

\n
단계 4: 분수를 약분하면 x = -\frac{15}{4}가 됩니다.\n


<IPython.core.display.Math object>

\n
최종 답:\n\n


<IPython.core.display.Math object>

## Pydantic 라이브러리의 `BaseModel`

In [7]:
from pydantic import BaseModel

# Pydantic 모델을 사용하여 응답 구조를 정의한다.
class MathReasoning(BaseModel):
    class Step(BaseModel):
        explanation: str
        output: str

    steps: list[Step]
    final_answer: str

# Pydantic 모델을 사용하여 응답을 파싱하는 함수를 재정의한다.
def get_math_solution(question: str):
    completion = client.beta.chat.completions.parse(
        model=MODEL,
        messages=[
            {"role": "system", "content": dedent(math_tutor_prompt)},
            {"role": "user", "content": question},
        ],
        response_format=MathReasoning, # JSON 스키마 대신 Pydantic 모델을 전달한다.
    )

    return completion.choices[0].message

In [8]:
# get_math_solution 함수를 호출하고 결과를 파싱한다.
result = get_math_solution(question).parsed

In [9]:
# 파싱된 결과를 출력한다.
print(result.steps)
print("최종 답:")
print(result.final_answer)

[Step(explanation='방정식 8x + 7 = -23에서 양변에서 7을 빼서 8x만 남기기 위해 양변에서 7을 뺍니다.', output='8x + 7 - 7 = -23 - 7'), Step(explanation='양변 계산 후 방정식은 8x = -30이 됩니다.', output='8x = -30'), Step(explanation='x의 계수 8로 양변을 나누어 x의 값을 구합니다.', output='x = \\frac{-30}{8}'), Step(explanation='분수를 기약분수로 단순화합니다.', output='x = -\\frac{15}{4}')]
최종 답:
x = -\frac{15}{4}


## 예제 2: 텍스트 요약

In [12]:
# 분석할 파일 경로 목록을 정의한다.
articles = [
    "../dataset/mcp-acp-a2a.md",
]

In [13]:
# 파일 경로에서 기사 내용을 읽는 함수를 정의한다.
def get_article_content(path):
    with open(path, 'r') as f:
        content = f.read()
    return content

# 각 기사 파일의 내용을 읽어온다.
content = [get_article_content(path) for path in articles]

In [23]:
# 텍스트 요약을 위한 시스템 프롬프트를 정의한다.
summarization_prompt = '''
    발명에 관한 기사 내용이 제공될 것이다.
    제공된 스키마에 따라 기사를 요약하는 것이 목표다.
    매개변수에 대한 설명은 다음과 같다:
    - invented_year: 기사에서 논의된 발명이 발명된 연도
    - summary: 발명품이 무엇인지에 대한 한 문장 요약
    - inventors: 발명가의 전체 이름이 있는 경우 목록으로 표시하고, 그렇지 않으면 성만 표시하는 문자열 배열
    - concepts: 발명과 관련된 주요 개념 배열, 각 개념은 제목과 설명을 포함한다.
    - description: 발명품에 대한 간략한 설명dmf gksrmffh
'''

# 기사 요약을 위한 Pydantic 모델을 정의한다.
class ArticleSummary(BaseModel):
    invented_year: int
    summary: str
    inventors: list[str]
    description: str

    class Concept(BaseModel):
        title: str
        description: str

    concepts: list[Concept]

# 기사 텍스트를 받아 요약을 반환하는 함수를 정의한다.
def get_article_summary(text: str):
    completion = client.beta.chat.completions.parse(
        model=MODEL,
        temperature=0.2, # 응답의 일관성을 높이기 위해 온도를 낮게 설정한다.
        messages=[
            {"role": "system", "content": dedent(summarization_prompt)},
            {"role": "user", "content": text}
        ],
        response_format=ArticleSummary,
    )

    return completion.choices[0].message.parsed

In [24]:
summaries = []

# 각 문서에 대해 요약을 생성한다.
for i in range(len(content)):
    print(f"기사 #{i+1} 분석 중...")
    summaries.append(get_article_summary(content[i]))
    print("완료.")

기사 #1 분석 중...
완료.


In [25]:
# 요약 내용을 출력하는 함수를 정의한다.
def summary(summary):
    print(f"발명 연도: {summary.invented_year}\\n")
    print(f"요약: {summary.summary}\\n")
    print("발명가:")
    
    for i in summary.inventors:
        print(f"- {i}")
    print("\\n개념:")
    for c in summary.concepts:
        print(f"- {c.title}: {c.description}")
    print(f"\\n설명: {summary.description}")

In [26]:
# 생성된 요약들을 출력한다.
for i in range(len(summaries)):
    print(f"기사 {i}\\n")
    summary(summaries[i])

기사 0\n
발명 연도: 2024\n
요약: Model Context Protocol (MCP), Agent Communication Protocol (ACP), 그리고 Agent-to-Agent Protocol (A2A)는 AI 에이전트의 상호 운용성과 협업을 위한 세 가지 핵심 프로토콜로, 각각 외부 데이터 연결, 에이전트 간 통신, 분산 협업을 지원한다.\n
발명가:
- Anthropic
- Linux Foundation
- Google
\n개념:
- Model Context Protocol (MCP): AI 에이전트와 외부 데이터 소스 간 연결을 표준화하여 정보 사일로 문제를 해결하고 개발 효율성을 높이는 프로토콜로, Tools, Resources, Prompts 세 가지 구성 요소를 제공한다.
- Agent Communication Protocol (ACP): Linux Foundation이 관리하는 오픈 표준 프로토콜로, REST API 기반의 다중 에이전트 통신과 세션 상태 관리, 멀티파트 메시지 지원을 통해 에이전트 간 협업을 촉진한다.
- Agent-to-Agent Protocol (A2A): Google이 개발한 오픈 프로토콜로, JSON-RPC 2.0 기반 1 에이전트/서버 구조와 분산 상태 관리, 불변 아티팩트 관리를 통해 대규모 분산 에이전트 협업을 지원한다.
- 프로토콜 간 상호작용: MCP는 컨텍스트 확장 역할을, ACP와 A2A는 에이전트 간 협업 역할을 수행하며, 실제 환경에서는 세 프로토콜이 함께 사용되어 AI 에이전트 시스템의 복잡한 요구를 충족한다.
- 거버넌스 및 아키텍처 차이: ACP는 Linux Foundation의 벤더 중립적 오픈 거버넌스와 REST 기반 다중 에이전트 아키텍처를, A2A는 Google 주도의 오픈소스와 JSON-RPC 기반 1 에이전트/서버 구조를 채택하여 각각 다른 철학과 기술적 특성을 가진다.
\n설명: MCP는 AI 모델과 외부 데이터 소스 간의 연결을 표준화하여 컨텍스트 확장을 지원하며, J

## 예제 3: 사용자 입력에서 엔티티 추출하기

In [27]:
from enum import Enum
from typing import Union
import openai

# 제품 검색을 위한 시스템 프롬프트를 정의한다.
product_search_prompt = '''
    당신은 사용자를 위한 완벽한 매치를 찾는 데 특화된 의류 추천 에이전트다.
    사용자 입력과 함께 사용자 성별, 연령대, 계절과 같은 추가 컨텍스트가 제공될 것이다.
    당신은 사용자의 프로필과 선호도에 맞는 의류를 데이터베이스에서 검색하는 도구를 갖추고 있다.
    사용자 입력과 컨텍스트를 기반으로, 데이터베이스를 검색하는 데 사용할 매개변수의 가장 가능성 있는 값을 결정하라.

    웹사이트에서 사용 가능한 다양한 카테고리는 다음과 같다:
    - shoes: boots, sneakers, sandals
    - jackets: winter coats, cardigans, parkas, rain jackets
    - tops: shirts, blouses, t-shirts, crop tops, sweaters
    - bottoms: jeans, skirts, trousers, joggers

    다양한 색상이 있지만, 일반적인 색상 이름을 사용하도록 하라.
'''

# 의류 카테고리를 위한 Enum 클래스를 정의한다.
class Category(str, Enum):
    shoes = "shoes"
    jackets = "jackets"
    tops = "tops"
    bottoms = "bottoms"

# 제품 검색 매개변수를 위한 Pydantic 모델을 정의한다.
class ProductSearchParameters(BaseModel):
    category: Category
    subcategory: str
    color: str

# 사용자 입력과 컨텍스트를 받아 도구 호출을 반환하는 함수를 정의한다.
def get_response(user_input, context):
    response = client.chat.completions.create(
        model=MODEL,
        temperature=0, # 일관된 결과를 위해 온도를 0으로 설정한다.
        messages=[
            {
                "role": "system",
                "content": dedent(product_search_prompt)
            },
            {
                "role": "user",
                "content": f"CONTEXT: {context}\\n USER INPUT: {user_input}"
            }
        ],
        tools=[
            # Pydantic 모델을 사용하여 함수 도구를 정의한다.
            openai.pydantic_function_tool(ProductSearchParameters, name="product_search", description="제품 데이터베이스에서 일치하는 항목 검색")
        ]
    )

    return response.choices[0].message.tool_calls

In [28]:
# 테스트를 위한 예제 입력 데이터 목록을 정의한다.
example_inputs = [
    {
        "user_input": "새 코트를 찾고 있다. 나는 항상 추위를 타서 따뜻한 것이 필요하다. 내 눈 색깔과 어울리는 것이면 좋겠다.",
        "context": "성별: 여성, 연령대: 40-50, 외모: 파란 눈"
    },
    {
        "user_input": "이번 여름에 스코틀랜드로 트레킹을 간다. 비가 올 것 같다. 무언가 찾는 것을 도와달라.",
        "context": "성별: 남성, 연령대: 30-40"
    },
    # ... (다른 예제들도 동일하게 번역)
]

In [34]:
# 도구 호출 결과를 출력하는 함수를 정의한다.
def tool_call(user_input, context, tool_call):
    args = tool_call[0].function.arguments
    print(f"입력: {user_input}\\n\\n컨텍스트: {context}\\n")
    print("제품 검색 인자:")
    for key, value in json.loads(args).items():
        print(f"{key}: '{value}'")
    print("\\n\\n")

In [36]:
for ex in example_inputs:
        tool_calls = get_response(ex["user_input"], ex["context"])
        print_tool_call(ex["user_input"], ex["context"], tool_calls)

입력: 새 코트를 찾고 있다. 나는 항상 추위를 타서 따뜻한 것이 필요하다. 내 눈 색깔과 어울리는 것이면 좋겠다.\n\n컨텍스트: 성별: 여성, 연령대: 40-50, 외모: 파란 눈\n
제품 검색 인자:
category: 'jackets'
subcategory: 'winter coats'
color: 'blue'
\n\n
입력: 이번 여름에 스코틀랜드로 트레킹을 간다. 비가 올 것 같다. 무언가 찾는 것을 도와달라.\n\n컨텍스트: 성별: 남성, 연령대: 30-40\n
제품 검색 인자:
category: 'jackets'
subcategory: 'rain jackets'
color: 'blue'
\n\n
